<a href="https://colab.research.google.com/github/jodfedlet/TCC/blob/main/TCCII/Twitter/preprocessing_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importando/instalando bibliotecas

In [1]:
!pip install unidecode
!python3 -m spacy download pt
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.2 MB 12.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

import pandas as pd
import numpy as np
import nltk
import re
import spacy
import gensim
import matplotlib as mpl
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from google.colab import drive
drive.mount("/content/drive")
nltk.download('all-corpora')
nltk.download('punkt')
nltk.download('rslp')

from nltk.corpus import stopwords
from unidecode import unidecode
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk import regexp_tokenize
from gensim.models import Phrases


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Download

# Carregamento dos tweets

In [3]:
try:
  tweets_orig = pd.read_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/all_tweets_with_retweets.csv', lineterminator='\n', index_col=0)
except:
  print("Não foi possível carrargar o arquivo")

In [4]:
tweets_orig.shape

(185348, 4)

#Estatística dos tweets

In [48]:
def tweets_stats(df, option):
  df['datetime'] = pd.to_datetime(df.datetime, format='%Y-%m-%d %H:%M:%S')

  def get_df_len__count_words__mean(df):
    count_all_tweets = len(df)

    if option == 'pre':
      df_text = df['text_preprocessed']
    elif option =='big':
       df_text = df['bigrams_text']
    else:
       df_text = df['text']

    df['count_word'] = df_text.str.split().str.len()
    count_all_words = df['count_word'].sum()
    mean = round(count_all_words / count_all_tweets, 2)
    return (df, count_all_tweets, count_all_words, mean)

  #data by year
  def plot_by_year(): 
    by_year = df.groupby([df['datetime'].dt.year.rename('Year')]).agg({'tweet_id':['count'], 'count_word': ['sum']}).reset_index()
    by_year.columns = ['Year', 'Count', 'Sum']
    by_year = by_year.assign(Mean_words = lambda x: round(x['Sum'] / x['Count'], 2))
    print(by_year)

  #by year month
  def plot_by_year_and_month():
    by_year_month = df.groupby([df['datetime'].dt.year.rename('Year'), df['datetime'].dt.month.rename('Month')]).agg({'tweet_id':['count'], 'count_word': ['sum']}).reset_index()
    by_year_month.columns = ['Year', 'Month', 'Count', 'Sum']
    by_year_month = by_year_month.assign(Mean_words = lambda x: round(x['Sum'] / x['Count'], 2))
    print(by_year_month)
   
  df, df_len, count_words, mean_words  = get_df_len__count_words__mean(df)
  print('Total: {0}, Count word: {1}, Mean words: {2}'.format(df_len,count_words, mean_words))
  print()
  print('*****Agrupado por ano****')
  plot_by_year()
  print()
  print('*****Agrupado por mês e ano****')
  plot_by_year_and_month()

###Estatística dos tweets antes do pré-processamento:

In [6]:
tweets_stats(tweets_orig, 'ori')

Total: 185348, Count word: 4542561, Mean words: 24.51

*****Agrupado por ano****
   Year  Count      Sum  Mean_words
0  2018  23691   531047       22.42
1  2019  29739   742523       24.97
2  2020  56375  1317064       23.36
3  2021  62625  1621237       25.89
4  2022  12918   330690       25.60

*****Agrupado por mês e ano****
    Year  Month  Count     Sum  Mean_words
0   2018      1   2564   56315       21.96
1   2018      2   2525   54548       21.60
2   2018      3   1300   28565       21.97
3   2018      4   1298   29096       22.42
4   2018      5   2613   57054       21.83
5   2018      6   1634   37871       23.18
6   2018      7   2412   48864       20.26
7   2018      8   1978   44443       22.47
8   2018      9   1558   36399       23.36
9   2018     10   1909   43484       22.78
10  2018     11   2170   55271       25.47
11  2018     12   1730   39137       22.62
12  2019      1   2532   65964       26.05
13  2019      2   4208  105975       25.18
14  2019      3   2106   

#Pegar os stop words da língua portuguesa usando NLTK

In [7]:
stop_words = set(stopwords.words('portuguese'))

In [8]:
type(stop_words)


set

In [9]:
stop_words

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'até',
 'com',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'entre',
 'era',
 'eram',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estas',
 'estava',
 'estavam',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'forem',
 'formos',
 'fosse',
 'fossem',
 'fui',
 'fôramos',
 'fôssemos',
 'haja',
 'hajam',
 'hajamos',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houver',
 'houvera',
 'houveram',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houvermos',
 'houverá',
 'houverão',
 'houveríamos',
 'houvesse',


#Pré-processamento dos tweets

###Função para limpar os tweets

In [10]:
def cleaning(tweet):
  
  #remover pontuações
  def remove_punctuation(tweet):
      return re.sub(r'[^\w\s]','',tweet) 

  #remocao de plavras com mais de 3 ocorrencias seguidas
  def mais_de_3_ocorrencias_seguidas(word):  
    let_ant = word[0]
    count = 0
    mais_de_3 = False;
    for w in word:
      count = count + 1 if w == let_ant else 1
      let_ant = w  
      if count > 3: 
        mais_de_3 = True 
        break  
    return mais_de_3   

  #remover acentuações
  def remove_emphasis(token):
        return unidecode(remove_punctuation(token))

  stop_words_list = [remove_emphasis(st_word) for st_word in stop_words]
 
  #conversão de string para minúsculas, remoção de pontuação, palavras que iniciam com @, #, stopwords ou com tamanho < 2 e se forem diferentes de ht.
  def remove_undesirable_words(tweet):
      new_list_ = []
      tweet = ' '.join(str(word).lower() for word in tweet.split() if not word.startswith('@') and not word.startswith('#'))
     
      for token in wordpunct_tokenize(tweet):
         if token in stop_words_list or (len(token) < 3 and token != 'ht') or mais_de_3_ocorrencias_seguidas(token): continue
         new_list_.append(remove_emphasis(token))
      return ' '.join(new_list_)

  #remover urls
  def remove_url(tweet_without_sw):
      return re.sub(r"http\S+", "", tweet_without_sw)

  #remover palavras que contem ou que são números
  def remove_numeric(tweet):
    return re.sub(r'\d+', '', tweet)

  tweet = remove_url(tweet)
  tweet = remove_numeric(tweet)
  tweet = remove_undesirable_words(tweet)
  return tweet 



###Exemplo de tweet antes da aplicação da função de limpeza

In [11]:
string_example = "No Rio de Janeiro, em 2021, <p> o nao @senhor vai #investir, em 2022, na educação da Venezuela, Argentina, Haiti,cuba.e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso ?????. https://t.co/jmVSYbsG2X"

In [12]:
filtered_tweet = cleaning(string_example)

###Exemplo de tweet depois da aplicação da função de limpeza

In [13]:
filtered_tweet

'rio janeiro vai educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez'

###Aplicação da função do pré-processamento nos tweets

In [14]:
tweets_pre_proc = tweets_orig.copy()
tweets_pre_proc['text_preprocessed'] = tweets_orig['text'].map(lambda tweet: cleaning(tweet))

In [15]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

In [16]:
tweets_pre_proc.text_preprocessed.head().tolist()


['paqueta haitiano',
 'haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar ']

###Remover tweets com menos de 3 palavras

In [17]:
tweets_pre_proc = tweets_pre_proc.loc[tweets_pre_proc['text_preprocessed'].str.split().str.len() > 3]

In [18]:
tweets_pre_proc.shape

(171087, 6)

###Remover os tweets duplicados

In [19]:
tweets_pre_proc.text.head().tolist()

['@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???',
 '@alice_mader9 kkkkkk e era 2 haitiano, n entendi nada q falavam']

In [20]:
tweets_pre_proc.drop_duplicates(subset = ["text_preprocessed"], keep='first', inplace=True)

###Quantidade de tweets após a remoção dos tweets duplicados

In [21]:
tweets_pre_proc.shape

(150264, 6)

In [22]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

In [23]:
tweets_pre_proc.text.head().tolist()

['@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???',
 '@alice_mader9 kkkkkk e era 2 haitiano, n entendi nada q falavam']

###Estatística dos tweets após o pré-processamento:

In [24]:
tweets_stats(tweets_pre_proc, 'pre')

Total: 150264, Count word: 2137665, Mean words: 14.23

*****Agrupado por ano****
   Year  Count     Sum  Mean_words
0  2018  18202  247212       13.58
1  2019  24449  364905       14.93
2  2020  47530  650724       13.69
3  2021  48733  704287       14.45
4  2022  11350  170537       15.03

*****Agrupado por mês e ano****
    Year  Month  Count     Sum  Mean_words
0   2018      1   1957   25945       13.26
1   2018      2   1836   24396       13.29
2   2018      3   1026   13485       13.14
3   2018      4    984   13494       13.71
4   2018      5   2095   26978       12.88
5   2018      6   1366   18525       13.56
6   2018      7   1861   23147       12.44
7   2018      8   1520   21052       13.85
8   2018      9   1195   16893       14.14
9   2018     10   1412   19919       14.11
10  2018     11   1713   26193       15.29
11  2018     12   1237   17185       13.89
12  2019      1   2089   32232       15.43
13  2019      2   3553   53191       14.97
14  2019      3   1700   25849 

###Lematização e stemização

In [25]:
pln = spacy.load('pt')

###Lematização

In [26]:
string_lem_example = 'lixo tiver sou capacetes brancos ajudaram haiti pare besteira ajudou haiti fim exercito capacetes brancos resolvem problema presos campos concentracao argentina criou'

In [27]:
def lemmatization(tweet):
  document = pln(tweet)
  return ' '.join([token.lemma_ for token in document])

In [28]:
tweet_lem_example = lemmatization(string_lem_example)

In [29]:
tweet_lem_example

'lixar ter ser capacete branco ajudar haiti parir besteira ajudar haiti fim exercitar capacete branco resolver problema preso campo concentracao argentino criar'

In [30]:
tweets_pre_proc['text_lematizados'] = tweets_pre_proc['text_preprocessed'].map(lambda tweet: lemmatization(tweet))

In [31]:
tweets_pre_proc.text_lematizados.head().tolist()

['haiti desgraca fato erro anular acerto falir sobrar guerra ucrania vir posicionar bem criticar posturar zelensky',
 'senhor ir investir educacao venezuela argentino haiti cubar etc paises comunista porque aqui brasil senhor nunca fazer',
 'nado mao voces virar misturar desertar saara haiti',
 'entrar brics invadir haiti mandar bush lula dizer ir fortalecer brics fazer palestrar parlamentar europeu este tratar russia chino inimigo mortal ir funcionar',
 'haitiano entender nado falar']

###Bigramas

In [32]:
def bigrams(df):
  docs = [doc.split() for doc in df.text_lematizados.tolist()]
  bigram = Phrases(docs, min_count=3)
  for i in range(len(docs)):
    docs[i] = ' '.join(bigram[docs[i]])
  return docs  

In [33]:
tweets_pre_proc['bigrams_text'] = bigrams(tweets_pre_proc)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [34]:
tweets_pre_proc.bigrams_text.head().tolist()

['haiti desgraca fato erro anular acerto falir sobrar guerra_ucrania vir posicionar bem criticar posturar zelensky',
 'senhor ir investir_educacao venezuela argentino haiti cubar etc paises comunista porque aqui brasil senhor nunca fazer',
 'nado mao voces virar_misturar desertar_saara haiti',
 'entrar brics invadir haiti mandar bush lula dizer ir fortalecer brics fazer palestrar parlamentar europeu este tratar russia_chino inimigo mortal ir funcionar',
 'haitiano entender_nado falar']

In [49]:
tweets_stats(tweets_pre_proc, 'big')

Total: 150264, Count word: 1938860, Mean words: 12.9

*****Agrupado por ano****
   Year  Count     Sum  Mean_words
0  2018  18202  225518       12.39
1  2019  24449  330502       13.52
2  2020  47530  597148       12.56
3  2021  48733  630554       12.94
4  2022  11350  155138       13.67

*****Agrupado por mês e ano****
    Year  Month  Count     Sum  Mean_words
0   2018      1   1957   23545       12.03
1   2018      2   1836   21993       11.98
2   2018      3   1026   12408       12.09
3   2018      4    984   12496       12.70
4   2018      5   2095   24680       11.78
5   2018      6   1366   17000       12.45
6   2018      7   1861   21067       11.32
7   2018      8   1520   19345       12.73
8   2018      9   1195   15521       12.99
9   2018     10   1412   18146       12.85
10  2018     11   1713   23670       13.82
11  2018     12   1237   15647       12.65
12  2019      1   2089   29248       14.00
13  2019      2   3553   48170       13.56
14  2019      3   1700   23491  

In [50]:
tweets_pre_proc.bigrams_text.tolist()

['haiti desgraca fato erro anular acerto falir sobrar guerra_ucrania vir posicionar bem criticar posturar zelensky',
 'senhor ir investir_educacao venezuela argentino haiti cubar etc paises comunista porque aqui brasil senhor nunca fazer',
 'nado mao voces virar_misturar desertar_saara haiti',
 'entrar brics invadir haiti mandar bush lula dizer ir fortalecer brics fazer palestrar parlamentar europeu este tratar russia_chino inimigo mortal ir funcionar',
 'haitiano entender_nado falar',
 'adorar caro cagar general_heleno primar pro haiti sempre mandar foto pra pra perguntar assim n_aquele tempo',
 'jane angeli junqueira advogar maringaense cafe_manha oferecer embaixada haiti kubitschek plaza hotel brasilia anfitria embaixador haiti brasil rachel coupau algum',
 'conseguir motorista haitiano fmz merda amanhar ir fazer correr tentar segundo ver sepa pros creditar voltar dps',
 'votar santo_cruz nao tera aceitar convidar dizer participar golpe alar cruz nao morar nunca fazer brasil bem mor

###Stematização ( não será usado na experimentação)

In [36]:
#string_stem_example = 'capacetes brancos ajudaram haiti pare besteira ajudou haiti fim exercito capacetes brancos resolvem problema presos campos concentracao argentina criou'

In [37]:
# def stemming(tweet):
#   document = pln(tweet)
#   stemmer = nltk.stem.RSLPStemmer()
#   return ' '.join([stemmer.stem(token.text) for token in document])

In [38]:
#string_stem_example = stemming(string_stem_example)

In [39]:
# string_stem_example

In [40]:
# tweets_stem = tweets_pre_proc.copy()
# tweets_stem['text'] = tweets_pre_proc['text'].map(lambda tweet: stemming(tweet))

In [41]:
#tweets_stem.text.head().tolist()

#Comparação de tweets originais, pré-processados, lematizados, **bigramados e **stematizados

###Tweets originais

In [42]:
tweets_orig.text.head().tolist()

['@geglobo Paquetá é haitiano',
 '@BetoLuiz_RU @jonesmanoel_PCB @deccache O Haiti foi uma desgraça de fato.Mas os erros nao anulam os acertos.E quando ele fala sobre a guerra na Ucrania ele vem se posicionando muito bem,criticando a postura do zelensky',
 'O senhor vai investir na educação da Venezuela, Argentina, Haiti,cuba, e etc países comunistas?, porque aqui no Brasil o Senhor nunca fez isso. https://t.co/jmVSYbsG2X',
 '@rsallesmma Que nada, na mão de vocês já tá virando uma mistura de deserto do Saara com Haiti',
 '@JoppertLeonardo @jonesmanoel_PCB @deccache Entramos nos BRICS, mas invadimos o Haiti a mando do Bush!\n\nLula diz que vai fortalecer os brics mas faz palestras no Parlamento Europeu que está tratando Rússia e china como inimigos mortais, como isso vai funcionar???']

### Tweets pré-processados

In [43]:
tweets_pre_proc.text_preprocessed.head().tolist()

['haiti desgraca fato erros anulam acertos fala sobre guerra ucrania vem posicionando bem criticando postura zelensky',
 'senhor vai investir educacao venezuela argentina haiti cuba etc paises comunistas porque aqui brasil senhor nunca fez',
 'nada mao voces virando mistura deserto saara haiti',
 'entramos brics invadimos haiti mando bush lula diz vai fortalecer brics faz palestras parlamento europeu esta tratando russia china inimigos mortais vai funcionar ',
 'haitiano entendi nada falavam']

###Tweets lematizados

In [44]:
tweets_pre_proc.text_lematizados.head().tolist()

['haiti desgraca fato erro anular acerto falir sobrar guerra ucrania vir posicionar bem criticar posturar zelensky',
 'senhor ir investir educacao venezuela argentino haiti cubar etc paises comunista porque aqui brasil senhor nunca fazer',
 'nado mao voces virar misturar desertar saara haiti',
 'entrar brics invadir haiti mandar bush lula dizer ir fortalecer brics fazer palestrar parlamentar europeu este tratar russia chino inimigo mortal ir funcionar',
 'haitiano entender nado falar']

In [52]:
tweets_pre_proc.bigrams_text.head().tolist()

['haiti desgraca fato erro anular acerto falir sobrar guerra_ucrania vir posicionar bem criticar posturar zelensky',
 'senhor ir investir_educacao venezuela argentino haiti cubar etc paises comunista porque aqui brasil senhor nunca fazer',
 'nado mao voces virar_misturar desertar_saara haiti',
 'entrar brics invadir haiti mandar bush lula dizer ir fortalecer brics fazer palestrar parlamentar europeu este tratar russia_chino inimigo mortal ir funcionar',
 'haitiano entender_nado falar']

###Tweets com bigramas

###Tweets stematizados

In [45]:
#tweets_stem.text.head().tolist()

##Criando o arquivo .csv dos tweets

In [46]:
df = pd.DataFrame(tweets_pre_proc)
df.to_csv('/content/drive/My Drive/Colab Notebooks/TCC/csv/pre_processed_tweets.csv')